# SPEEDUP

In [1]:
from hamiltonian import Hamiltonian, HamiltonianH2, HamiltonianW

In [2]:
lih = Hamiltonian('LiH', 1.5)
beh2 = Hamiltonian('BeH2', 1.3)

h2_jw = HamiltonianH2('jw')
h2_parity = HamiltonianH2('parity')
h2_bk = HamiltonianH2('bk')

water_jw = HamiltonianW('jw')
water_parity = HamiltonianW('parity')
water_bk = HamiltonianW('bk')

## sparse.py

In [112]:
import numpy as np
from scipy import sparse
from scipy.sparse.linalg import eigsh

row = np.array([0, 1])
col = np.array([0, 1])
data = np.array([1+0j, 1+0j])
id = sparse.csr_matrix((data, (row, col)), shape=(2, 2))
x = sparse.csr_matrix(([1+0j, 1+0j], ([0, 1], [1, 0])), shape=(2, 2))
y = sparse.csr_matrix(([0-1j, 0+1j], ([0, 1], [1, 0])), shape=(2, 2))
z = sparse.csr_matrix(([1+0j, -1+0j], ([0, 1], [0, 1])), shape=(2, 2))

_to_matrix = {'I': id, 'X': x, 'Y': y, 'Z': z}


def to_matrix(pauli_string):
    p0 = pauli_string[0]
    mat = _to_matrix[p0]
    for p in pauli_string[1:]:
        mat = sparse.kron(mat, _to_matrix[p], format="csr")
    return mat


def matrix(pauli_rep):
    n = pauli_rep.num_qubits
    mat = sparse.csr_matrix((2**n, 2**n), dtype=np.complex128)
    for pauli_string, coefficient in pauli_rep.dic.items():
        mat += coefficient * to_matrix(pauli_string)
    return mat


def ground(pauli_rep):
    mat = matrix(pauli_rep)
    evals, evecs = eigsh(mat, which='SA')
    # SA looks for algebraically small evalues
    index = np.argmin(evals)
    return evals[index], evecs[:, index]


def energy(pauli_rep, state, tol=1e-6):
    '''
    Directly calculate energy by tracing state over Pauli kronecker product.
    Return real float.
    '''
    state_dual = np.conjugate(state)
    mat = matrix(pauli_rep)
    trace = np.dot(state_dual, mat * state)
    assert abs(trace.imag) < tol
    return np.real(trace)

## variance.py

In [88]:
import numpy as np

def f_individual(q, r, βi):
    assert q in ['I', 'X', 'Y', 'Z']
    assert r in ['I', 'X', 'Y', 'Z']
    assert len(βi) == 3
    if q == 'I' or r == 'I':
        return 1.0
    elif q == r:
        dic = {'X': 0, 'Y': 1, 'Z': 2}
        if βi[dic[q]] == 0.0:
            return 0.0
        return (βi[dic[q]])**(-1)
    else:
        return 0.0


def f_string(Q, R, β):
    assert len(Q) == len(R)
    assert len(Q) == len(β.keys())
    prod = 1.0
    for i in range(len(Q)):
        # qiskit ordering
        prod *= f_individual(Q[i], R[i], β[(len(Q)-1)-i])
    return prod


def pauli_multiply_individual(q, r):
    assert q in ['I', 'X', 'Y', 'Z']
    assert r in ['I', 'X', 'Y', 'Z']
    if q == r:
        # return q*r
        return 'I'
    else:
        # at least one of q or r is the identity, the other one should be returned
        if q != 'I':
            return q
        else:
            return r
    # error if I arrive here


def pauli_multiply_string(Q, R):
    assert len(Q) == len(R)
    string = ''
    for i in range(len(Q)):
        string += pauli_multiply_individual(Q[i], R[i])
    return string


def variance_local(pauli_rep, energy, state, β):
    var = 0.0

    tallyf = 0
    tallyQ = 0
    for Q, alphaQ in pauli_rep.dic.items():
        tallyQ +=1
        if Q == 'I' * pauli_rep.num_qubits:
            continue
        tallyR = 0
        for R, alphaR in pauli_rep.dic.items():
            tallyR+=1
            if R == 'I' * pauli_rep.num_qubits:
                continue
            f = f_string(Q, R, β)
            if f == 0.0:
                tallyf +=1
                continue
            # else, need to calculate < state | PQ | state >
            QR = pauli_multiply_string(Q, R)
            #QRmat = to_matrix(QR)
            #tr_rho_QR = np.dot(np.conjugate(state), QRmat * state).real

            #var += f * alphaQ * alphaR * tr_rho_QR

    energy_tf = pauli_rep.energy_tf(energy)
    var -= energy_tf**2
    print('tallyf: ', tallyf)
    print('pauli terms: ', len(pauli_rep.dic.keys()))
    print('fraction saved: ', tallyf / len(pauli_rep.dic.keys())**2)
    return var

In [31]:
pr = lih.pauli_rep
energy, state = pr.ground()
β = pr.local_dists_uniform()

%time variance_local(pr, energy, state, β)

tallyf:  7296
pauli terms:  100
fraction saved:  0.7296
CPU times: user 2.1 s, sys: 888 µs, total: 2.1 s
Wall time: 2.1 s


2.2307301610389274

In [32]:
pr = beh2.pauli_rep
energy, state = pr.ground()
β = pr.local_dists_uniform()

%time variance_local(pr, energy, state, β)

tallyf:  18992
pauli terms:  165
fraction saved:  0.6975941230486685
CPU times: user 11 s, sys: 9.32 ms, total: 11 s
Wall time: 11 s


18.179850794404423

In [33]:
pr = h2_bk.pauli_rep
energy, state = pr.ground()
β = pr.local_dists_uniform()

%time variance_local(pr, energy, state, β)

tallyf:  24948
pauli terms:  185
fraction saved:  0.7289408327246165
CPU times: user 17.5 s, sys: 113 ms, total: 17.7 s
Wall time: 16.8 s


168.8660032440544

In [66]:
184**2-24948

8908

In [34]:
from numpy import random

In [40]:
def random_pauli(pauli_rep):
    num_keys = len(pauli_rep.dic.keys())
    index_random = random.choice(num_keys)
    for index, pauli in enumerate(pauli_rep.dic.keys()):
        if index == index_random:
            return pauli
        pass
    pass

In [ ]:
pr = lih.pauli_rep
pauli = random_pauli(pr)





In [62]:
def foo():
    pass

import cProfile
cProfile.run('foo()')

         4 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-62-dd443bca37ae>:1(foo)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [85]:
pr = water_jw.pauli_rep
%time energy, state = pr.ground()
β = pr.local_dists_uniform()

CPU times: user 20.9 s, sys: 2.33 s, total: 23.3 s
Wall time: 9.75 s


In [86]:
%time pr.local_dists_optimal()

/opt/anaconda3/envs/qiskit18/lib/python3.8/site-packages/scipy/optimize/_hessian_update_strategy.py:183: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


CPU times: user 6min 50s, sys: 1min 3s, total: 7min 53s
Wall time: 59.1 s


{0: [0.43257858556565915, 0.43257858722135467, 0.1348428272129862],
 1: [0.24098007537433425, 0.2409800708577077, 0.518039853767958],
 2: [0.13870541903721406, 0.13870542588015417, 0.7225891550826317],
 3: [0.13957872152763764, 0.13957871134125624, 0.7208425671311062],
 4: [0.16984472160573807, 0.1698447243114196, 0.6603105540828423],
 5: [0.2675208449775755, 0.2675208434379076, 0.464958311584517],
 6: [0.11647067744295292, 0.11647067819430176, 0.7670586443627454],
 7: [0.43257857708372555, 0.43257860312684154, 0.13484281978943294],
 8: [0.24098005034367995, 0.24098008250406633, 0.5180398671522537],
 9: [0.13870542248842968, 0.13870542724539398, 0.7225891502661764],
 10: [0.13957872657623138, 0.13957872195005588, 0.7208425514737127],
 11: [0.16984475333796817, 0.16984473040786618, 0.6603105162541657],
 12: [0.2675208758292949, 0.26752084565650264, 0.46495827851420257],
 13: [0.11647066971784335, 0.1164706702788042, 0.7670586600033525]}

In [92]:
%time variance_local(pr, energy, state, β)

tallyf:  853052
pauli terms:  1086
fraction saved:  0.7232956530291776
CPU times: user 6.81 s, sys: 5.28 ms, total: 6.82 s
Wall time: 6.82 s


-804.0909451502213

In [117]:
pauli_string = random_pauli(pr)
%time to_matrix(pauli_string)

CPU times: user 7.28 ms, sys: 1.64 ms, total: 8.93 ms
Wall time: 7.5 ms


<16384x16384 sparse matrix of type '<class 'numpy.complex128'>'
	with 16384 stored elements in Compressed Sparse Row format>

For water there are 1086 pauli terms and 853052 pairs give f=0. We therefore call `sparse.kron` 324173 times.

Each `sparse.kron` call takes 5~10 milli seconds.

The expected time is therefore on the order 30~45 minutes. In reality, it is more like 25 minutes.

the following profiling was done with H2 pauli representation

In [64]:
cProfile.run('variance_local(pr, energy, state, β)')

tallyf:  24948
pauli terms:  185
fraction saved:  0.7289408327246165
         35016101 function calls (34891389 primitive calls) in 25.750 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   374136    0.174    0.000    0.530    0.000 <__array_function__ internals>:2(can_cast)
     8908    0.006    0.000    0.023    0.000 <__array_function__ internals>:2(dot)
   124712    0.072    0.000    0.164    0.000 <__array_function__ internals>:2(empty_like)
   187068    0.105    0.000    0.257    0.000 <__array_function__ internals>:2(ndim)
   187068    0.124    0.000    0.174    0.000 <frozen importlib._bootstrap>:389(parent)
     8908    0.209    0.000   25.242    0.003 <ipython-input-3-96df486656a9>:16(to_matrix)
    33856    0.134    0.000    0.273    0.000 <ipython-input-30-823ac56a38c8>:18(f_string)
    71264    0.015    0.000    0.015    0.000 <ipython-input-30-823ac56a38c8>:28(pauli_multiply_individual)
   270848    0.089    0